## Sample Loops 
### Do not run this! These are just samples you can repurpose.
### T-SQL, PowerShell

In [0]:
--SQLCMD
--:setvar rpl localhost
--:setvar ag AG1

USE [master]
go
if exists (select 1 from sys.availability_databases_cluster inner join sys.databases on database_name = name)
begin
	declare @sql table (smt nvarchar(max))
	declare @cnt int, @smt nvarchar(max)

	insert @sql 
	select 'ALTER AVAILABILITY GROUP [$(ag)] REMOVE DATABASE ['+name+'];' 
	from sys.databases where database_id>4 and name not like 'dist%' and name <> 'Claims' --in ('$(dbs)')

	select @cnt=count(*) from @sql

	while @cnt>0
	begin 
		select top 1 @smt=smt from @sql --order by smt; 
		print @smt; exec sp_executesql @smt	
		delete from @sql where @smt=smt
		set @cnt = @cnt - 1
	end
end 
go

In [0]:
--all else. DONOT RUN THIS!!!
declare @drop nvarchar(max)=null, @cnt int

select @cnt=count(*) from sys.databases where database_id >4 and name not like 'distribution%'

while @cnt>0
begin 
	select top 1 @drop='drop database ['+name+']' from sys.databases where database_id>4 and name not like 'distribution%'
	exec (@drop)
	select @cnt=count(*) from sys.databases where database_id >4 and name not like 'distribution%'
end

In [0]:
declare @name nvarchar(max)=null, @sql nvarchar(max), @count int
select @count=count(*) from sys.databases where state <>0

while @count>0
begin 
	select top 1 @name = name from sys.databases where state <>0
	set @sql = N'restore database ['+@name+'] with recovery'
	--print @sql; 
	exec sp_executesql @sql;
	select @count=count(*) from sys.databases where state <>0
end

In [0]:
--set owner
EXEC master.dbo.sp_MSforeachdb @command1 = N'
if ''?'' not in (''master'',''model'',''msdb'',''tempdb'',''distribution'',''distribution1'',''distribution2'',''SomeReadonlyDB'',''Skippa'') 
begin 
	exec ?.dbo.sp_changedbowner @loginame = sa, @map = false 
end'
go

--shrink log
if exists (select 1 from sys.databases where log_reuse_wait_desc = 'LOG_BACKUP' and database_id>4 and name not like 'dist%')
begin 
	exec msdb.dbo.sp_start_job 'Backup.TLogs'
	waitfor delay '00:00:30'
end
go
EXEC master.dbo.sp_MSforeachdb @command1 = 'exec (''USE ['' +''?''+ '']; IF DB_id(''''?'''')>4 BEGIN dbcc shrinkfile(2,1024) END'')';
go

In [0]:
set nocount on;

declare @p int
SELECT @p=count(*) FROM sys.dm_exec_requests r CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) a WHERE r.command like 'RESTORE%'

while @p > 0
begin 
	waitfor delay '00:05:00' --5 minutes
	SELECT @p=count(*) FROM sys.dm_exec_requests r CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) a WHERE r.command like 'RESTORE%'
end
go

In [0]:
--POST SSRS MIGRATION COPY
--DISABLE SSRS SUBSCRIPTION SQL JOBS
--4DisableSSRSSubscriptions.sql

USE msdb
GO
DECLARE @name sysname, @SQL NVARCHAR(MAX), @Count int

SELECT NAME [Enabled] FROM SYSJOBS J WHERE ENABLED=1 AND LEN(NAME)=36 ORDER  BY 1
SELECT @COUNT = COUNT(*) FROM SYSJOBS WHERE ENABLED=1 AND LEN(NAME)=36 --67

WHILE @Count > 0
	BEGIN
		SELECT TOP 1 @name = name FROM SYSJOBS WHERE ENABLED=1 AND LEN(NAME)=36 --67
		SET @SQL = 'EXEC msdb.dbo.sp_update_job  @job_name = '''+@name+''', @enabled = 0 ;'
		EXEC sp_executesql @SQL; PRINT @SQL;
		SELECT @Count = Count(*) FROM SYSJOBS WHERE ENABLED=1 AND LEN(NAME)=36 --67
	END

SELECT NAME [Enabled] FROM SYSJOBS J WHERE ENABLED=0 AND LEN(NAME)=36  ORDER  BY 1
go

USE MSDB;
GO
UPDATE MSDB.dbo.sysjobs SET Enabled = 0 WHERE len(name)=36;

In [0]:
:connect targethost
:setvar CHG  "228-2"
:setvar DISK ""
:setvar Database "RockOn"

--  +--------+
--  | Backup |
--  +--------+
set nocount on
use tempdb
go
declare @DB sysname, @SQL nvarchar(max), @Count int, @chg nvarchar(10), @timestamp nvarchar(25), @LastPath nvarchar(520)
set @chg = '$(CHG)' 
set @LastPath = '$(DISK)'

select @timestamp = replace(replace(replace(replace(convert(nvarchar(25), getdate(), 121),'-',''),':',''),'.',''),' ','_');

if ISNULL(@LastPath,'') = ''
begin
	select top 1 @LastPath=substring(bmf.physical_device_name,1,charindex('_backup',bmf.physical_device_name,0))
	from   		msdb.dbo.backupmediafamily as bmf
	inner join 	msdb.dbo.backupset as bs on bmf.media_set_id = bs.media_set_id  
	where  		bs.type = 'D' and bs.database_name in ('$(Database)')
	order by 	backup_start_date desc;
end;

if object_id('Tempdb..#dbs') is not null drop table #dbs;
select name into #dbs from sys.databases where name in ('$(Database)') --and database_id > 4

select @Count = count(*) from #dbs;

while @Count > 0
	begin
		select top 1 @DB = name FROM #dbs
		--set @DB = 'RockOn' --remove.
		set @SQL = 'backup database ['+@DB+'] to disk = '''+@LastPath+@DB+'_backup_'+@timestamp+'_pre'+@chg+'.bak'' 
			with noformat, noinit, skip, norewind, nounload, compression
			, encryption(algorithm = AES_256, server certificate = [BackupCertPK])
			, stats = 25;'
		exec sp_executesql @SQL; print @SQL;
		delete from #dbs --where name = @DB; --uncomment
		select @Count = Count(*) from #dbs;
	end
if object_id('Tempdb..#dbs') is not null drop table #dbs;
go
/*
--  +-------------------+
--  | RESTORE / REPLACE |
--  +-------------------+
set nocount on;
declare @kill nvarchar(max) = '', @LastPath nvarchar(520), @chg
set @chg = '$(CHG)' 
set @LastPath = '$(DISK)'

if exists (select 1 from sys.dm_exec_sessions where database_id  = db_id('$(Database)'))
begin
	select 	@kill = @kill + 'KILL ' + convert(varchar(5), session_id) + ';'
	from	sys.dm_exec_sessions
	where 	database_id  = db_id('$(Database)')

	exec sp_executesql @kill;
end
go

if db_id(N'$(Database)') is not null
begin 
	alter database [$(Database)] set single_user with rollback immediate;
	drop database [$(Database)] end
else begin
	restore database [$(Database)] from disk = 'C:\Deployments\RockOn.bak' with replace, file = 1,  
	move N'RockOn' to N'C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\RockOn.mdf', 
	move N'RockOn_log' to N'C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\RockOn_log.ldf',  
	nounload, stats = 25
go
*/

PowerShell

In [0]:
!powershell.exe
$Src = 'oldserver'
$Dst = 'newserver1','newserver2'
foreach ($d in $Dst)
{
    Start-DbaMigration -Source $Src -Destination $d  -DisableJobsOnDestination -Exclude Databases -Force
    Copy-DbaDatabaseMail -Source $Src -Destination $d -force
	Copy-DbaAgentOperator -Source $Src -Destination $d -force
	Copy-DbaAgentAlert -Source $Src -Destination $d -force
	Copy-DbaLogin -Source $Src -Destination $d -force
	Copy-DbaLinkedServer -Source $Src -Destination $d -force
    Copy-DbaExtendedEvent -Source $Src -Destination $d -force
    Copy-DbaAgentJob -Source $Src -Destination $d -DisableOnDestination -Force |ft -AutoSize
    Copy-DbaServerAudit -Source $Src -Destination $d -force
    Copy-DbaServerAuditSpecification -Source $Src -Destination $d -force
}